In [ ]:
'''Adapted from grid pattern'''

import itertools
import numpy as np

n = 4
f = 4*n-2

def get_edges_1():
    return [(i, i+1) for i in range(0, f-1, 12)] + \
    [(i, i+2) for i in range(2, f-2, 6)] + \
    [(i, i+2) for i in range(3, f-2, 6)]

def get_edges_2():
    return [(i, i+1) for i in range(4, f-1, 12)] + \
    [(i, i+2) for i in range(0, f-2, 6)] + \
    [(i, i+2) for i in range(1, f-2, 6)]

def get_edges_3():
    return [(i, i+1) for i in range(8, f-1, 12)] + \
    [(i, i+2) for i in range(4, f-2, 6)] + \
    [(i, i+2) for i in range(5, f-2, 6)]

def get_positive_faces():
    return list(range(0, f, 4)) + list(range(3, f, 4))

def get_negative_faces():
    return list(range(2, f, 4)) + list(range(1, f, 4))

edges_1 = get_edges_1()
edges_2 = get_edges_2()
edges_3 = get_edges_3()
positive_faces = get_positive_faces()
negative_faces = get_negative_faces()

# check for crossing violations
def check_crossings(edges, ordering):
    for e1, e2 in itertools.combinations(edges, 2):
        b1 = min(ordering[e1[0]], ordering[e1[1]])
        t1 = max(ordering[e1[0]], ordering[e1[1]])
        b2 = min(ordering[e2[0]], ordering[e2[1]])
        t2 = max(ordering[e2[0]], ordering[e2[1]])

        if b1 < b2 < t1 < t2 or b2 < b1 < t2 < t1:
            # print("CROSSING", e1, e2, b1, t1, b2, t2)
            return False
    return True
    
def check_all_crossings(assignment, ordering):
    if not check_crossings(edges_1, ordering) or not check_crossings(edges_2, ordering) or not check_crossings(edges_3, ordering):
        return False

    return True

def convertToMV(assign):
    edge_assign = {}
    edge_assign[(0,1)] = -1
    for i in range(len(assign)):
        base = 4*i
        flip = edge_assign[(base, base+1)]
        diff = list(itertools.combinations(range(6), 2))[assign[i]]
        if (0 in diff):
            flip = -1*flip
        if (1 in diff):
            edge_assign[(base, base+2)] = -1*flip
        else:
            edge_assign[(base, base+2)] = flip
            
        if (2 in diff):
            edge_assign[(base+2, base+4)] = -1*flip
        else:
            edge_assign[(base+2, base+4)] = flip
        
        if (3 in diff):
            edge_assign[(base+4, base+5)] = -1*flip
        else:
            edge_assign[(base+4, base+5)] = flip
        
        if (4 in diff):
            edge_assign[(base+3, base+5)] = -1*flip
        else:
            edge_assign[(base+3, base+5)] = flip
        
        if (5 in diff):
            edge_assign[(base+1, base+3)] = -1*flip
        else:
            edge_assign[(base+1, base+3)] = flip
            
    return edge_assign

def get_poset(assignment):
    poset = []
    # Loops over edges and adds them (using parity of edge and of face location)
    for edge in assignment:
        if (edge[0]%4 == 0 or edge[0]%4 == 3):
            if (assignment[edge] == -1):
                poset.append([edge[0], edge[1]])
            else:
                poset.append([edge[1], edge[0]])
        else:
            if (assignment[edge] == 1):
                poset.append([edge[0], edge[1]])
            else:
                poset.append([edge[1], edge[0]])
    
    return Poset((list(range(2 * n)), poset))

## Iterate over all possible linear extensions of the poset and check edges

def count(assignment):
    count = 0
    P = get_poset(assignment)
    for i in P.linear_extension().parent():
        if(check_all_crossings(assignment, np.argsort(i))):
            # print(i)
            count+= 1
    return count

def foldable(assignment):
    P = get_poset(assignment)
    perms = list(P.linear_extension().parent())
    np.random.shuffle(perms)
    for i in perms:
        if(check_all_crossings(assignment, np.argsort(i))):
            # print(i)
            return True
    return False
    
# for i in range(1, 5):
#     n = i
#     f = 4*n - 2
#     print(n)
#     with open("2x" + str(i) + "_triangle_count.txt", "w") as g:
#         for rep in list(itertools.product(list(range(15)), repeat=n-1)):
#             g.write(str(rep)+ " " + str(count(convertToMV(rep))) + "\n")

# for rep in list(itertools.product(list(range(15)), repeat=n-1)):
#     print(str(rep)+ " " + str(count(convertToMV(rep))))

In [131]:
len(get_poset(convertToMV([0, 0, 0])).linear_extension().parent())

12879990